### ew angles

In [28]:
from pathlib import Path
import pandas as pd
import numpy as np
from math import sqrt, acos, degrees

In [66]:
# пути 
#workdir = Path(r'D:\work\2022_m\220_man-van\test\phases')
workdir = Path(r'D:\work\2022_m\110_man-nps1\test\phases')

In [67]:
# read id 
#phase_tab = pd.read_excel(workdir / 'clearance_grd.xlsx', sheet_name='str_names', names=['id'], usecols='A')
phase_tab = pd.read_excel(workdir / 'all_in_one.xlsx', sheet_name='str_names', names=['id'], usecols='A')

In [68]:
phase_tab

,id
0,ПС_Мангазея
1,1
2,2
3,3
4,4
...,...
193,193
194,194
195,195
196,ПС_НПС-1


In [69]:
# read tk  

#tk_tab = pd.read_excel(workdir / 'clearance_grd.xlsx', sheet_name='attach', 
#                       names=['id', 'set', 'tx', 'ty', 'tz', 'mx', 'my', 'mz'], usecols='B,C,L:Q')

tk_tab = pd.read_excel(workdir / 'all_in_one.xlsx', sheet_name='attach', 
                       names=['id', 'set', 'tx', 'ty', 'tz', 'mx', 'my', 'mz'], usecols='B,C,L:Q')

In [70]:
tk_tab

,id,set,tx,ty,tz,mx,my,mz
0,ПС_Мангазея,1,557444.72,7388314.05,30.43,557455.34,7388308.16,31.44
1,ПС_Мангазея,2,557443.83,7388311.69,30.85,557454.83,7388306.14,33.68
2,ПС_Мангазея,3,557442.91,7388309.78,30.31,557454.21,7388305.16,29.76
3,ПС_Мангазея,4,557441.27,7388305.81,30.39,557453.08,7388300.06,29.85
4,ПС_Мангазея,5,557440.42,7388303.81,30.49,557452.83,7388298.91,32.95
...,...,...,...,...,...,...,...,...
1543,4a,12,558167.62,7388279.46,33.90,558249.12,7388286.90,27.11
1544,4a,13,558166.97,7388278.00,37.93,558248.62,7388285.39,31.50
1545,4a,14,558164.26,7388271.99,30.06,558246.59,7388279.25,25.41
1546,4a,15,558163.60,7388270.53,34.06,558246.09,7388277.74,29.38


In [71]:
def dist3d(a, b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dz = a[2] - b[2]
    
    return round(sqrt(dx*dx + dy*dy + dz*dz), 2)

In [72]:
def dist2d(a, b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    
    return round(sqrt(dx*dx + dy*dy), 2)

In [73]:
def ugol(a, b):
    # a is ew, b is down on wire
    
    # find point under a
    c = (a[0], a[1], b[2])
    
    ab = dist3d(a, b)
    ac = dist3d(a, c)
    
    return round(degrees(acos(ac/ab)), 1)

In [74]:
# сначала найдем глы на опоре.
# находим левую и правую верхние ТК, затем ближайшие к ним тросы
# расчет на две цепи при всегда верхних 3 и 6 сетах - т.к. это почти все опоры
# порталы и особые случаи считать вручную

ew_sets = [7, 8, 17, 18]

for n in range(len(phase_tab)):
#for n in range(3):
    idx = phase_tab.iloc[n].name
    structure = phase_tab.iloc[n, 0]
    select = tk_tab.loc[tk_tab['id'] == structure]
    
    setlist = list(select['set'].values)
    #print(setlist)

    #firstly check if there is any EW

    if not any(x in ew_sets for x in setlist):
        # no ew
        lt_ang = rt_ang = 0
    else:
        # here is some ew
           
        # find left and right points
        if len(setlist) > 10:
            left_t = (float(select.loc[select['set'] == 13].tx), 
                      float(select.loc[select['set'] == 13].ty),
                      float(select.loc[select['set'] == 13].tz)) 
            right_t = (float(select.loc[select['set'] == 16].tx), 
                      float(select.loc[select['set'] == 16].ty),
                      float(select.loc[select['set'] == 16].tz)) 
        else:
            if len(setlist) > 5:
                left_t = (float(select.loc[select['set'] == 3].tx), 
                          float(select.loc[select['set'] == 3].ty),
                          float(select.loc[select['set'] == 3].tz)) 
                right_t = (float(select.loc[select['set'] == 6].tx), 
                          float(select.loc[select['set'] == 6].ty),
                          float(select.loc[select['set'] == 6].tz)) 
            else:
                left_t = (float(select.loc[select['set'] == 2].tx), 
                          float(select.loc[select['set'] == 2].ty),
                          float(select.loc[select['set'] == 2].tz)) 
                right_t = (float(select.loc[select['set'] == 3].tx), 
                          float(select.loc[select['set'] == 3].ty),
                          float(select.loc[select['set'] == 3].tz)) 

        #print(left_t, right_t)

        # find all ew coords

        ew_points = []

        #print(list(select['set'].values))

        for s in ew_sets:
            if s in list(select['set'].values):
                ew_points.append((float(select.loc[select['set'] == s].tx), 
                      float(select.loc[select['set'] == s].ty),
                      float(select.loc[select['set'] == s].tz)))
        
        # find closest point ew
        if len(ew_points) == 1:
            left_ew = right_ew = ew_points[0]
        
        else:
            ldist = 100
            rdist = 100
            for ew in ew_points:
                if dist2d(left_t, ew) < ldist:
                    ldist = dist2d(left_t, ew)
                    left_ew = ew
               
                if  dist2d(right_t, ew) < rdist:
                    rdist = dist2d(right_t, ew)
                    right_ew = ew

        #print(left_ew, right_ew)
        
        # and now let calc angles
        lt_ang = ugol(left_ew, left_t)
        rt_ang = ugol(right_ew, right_t)
        
    phase_tab.loc[idx, 'ltow_ang'] = lt_ang
    phase_tab.loc[idx, 'rtow_ang'] = rt_ang



In [75]:
phase_tab

,id,ltow_ang,rtow_ang
0,ПС_Мангазея,39.7,63.2
1,1,33.2,29.0
2,2,22.6,22.6
3,3,29.4,32.4
4,4,31.2,36.6
...,...,...,...
193,193,13.1,13.1
194,194,12.9,13.1
195,195,27.3,18.1
196,ПС_НПС-1,59.3,77.5


### теперь тоже в середине пролета

In [76]:
ew_sets = [7, 8, 17, 18]

for n in range(len(phase_tab)):
#for n in range(3):
    idx = phase_tab.iloc[n].name
    structure = phase_tab.iloc[n, 0]

    select = tk_tab.loc[tk_tab['id'] == structure]
    
    setlist = list(select['set'].values)
    #print(setlist)

    #firstly check if there is any EW

    if not any(x in ew_sets for x in setlist):
        # no ew
        lm_ang = rm_ang = '-'

    else:
        # here is some ew
           
        # find left and right points
        if len(setlist) > 10:
            left_m = (float(select.loc[select['set'] == 13].mx), 
                      float(select.loc[select['set'] == 13].my),
                      float(select.loc[select['set'] == 13].mz)) 
            right_m = (float(select.loc[select['set'] == 16].mx), 
                      float(select.loc[select['set'] == 16].my),
                      float(select.loc[select['set'] == 16].mz)) 
        else:
            if len(setlist) > 5:
                left_m = (float(select.loc[select['set'] == 3].mx), 
                          float(select.loc[select['set'] == 3].my),
                          float(select.loc[select['set'] == 3].mz)) 
                right_m = (float(select.loc[select['set'] == 6].mx), 
                          float(select.loc[select['set'] == 6].my),
                          float(select.loc[select['set'] == 6].mz)) 
            else:
                left_m = (float(select.loc[select['set'] == 2].mx), 
                          float(select.loc[select['set'] == 2].my),
                          float(select.loc[select['set'] == 2].mz)) 
                right_m = (float(select.loc[select['set'] == 3].mx), 
                          float(select.loc[select['set'] == 3].my),
                          float(select.loc[select['set'] == 3].mz)) 

        #print(left_m, right_m)

        # find all ew coords

        ew_points = []

        #print(list(select['set'].values))

        for s in ew_sets:
            if s in setlist:
                if not float(select.loc[select['set'] == s].mx) == 0:
                    ew_points.append((float(select.loc[select['set'] == s].mx), 
                          float(select.loc[select['set'] == s].my),
                          float(select.loc[select['set'] == s].mz)))

        # find closest point ew
        if len(ew_points) == 0:
            lm_ang = rm_ang = '-' 

        else:
            if len(ew_points) == 1:
                left_ew = right_ew = ew_points[0]
            else:
                ldist = 100
                rdist = 100
                for ew in ew_points:
                    if dist2d(left_m, ew) < ldist:
                        ldist = dist2d(left_m, ew)
                        left_ew = ew

                    if  dist2d(right_m, ew) < rdist:
                        rdist = dist2d(right_m, ew)
                        right_ew = ew

            #print(left_ew, right_ew)

            # and now let calc angles
            lm_ang = ugol(left_ew, left_m)
            rm_ang = ugol(right_ew, right_m)
        
    phase_tab.loc[idx, 'lmid_ang'] = lm_ang
    phase_tab.loc[idx, 'rmid_ang'] = rm_ang


In [77]:
phase_tab

,id,ltow_ang,rtow_ang,lmid_ang,rmid_ang
0,ПС_Мангазея,39.7,63.2,20.8,49.0
1,1,33.2,29.0,28.6,27.7
2,2,22.6,22.6,28.2,30.0
3,3,29.4,32.4,37.8,37.5
4,4,31.2,36.6,58.9,83.9
...,...,...,...,...,...
193,193,13.1,13.1,11.8,12.1
194,194,12.9,13.1,18.7,15.4
195,195,27.3,18.1,14.5,6.1
196,ПС_НПС-1,59.3,77.5,-,-


In [78]:
# save tab
phase_tab.to_csv(workdir / 'ew_angles.txt', sep='\t', encoding = 'cp1251')